<a name="top"></a>*“Computing with semigroups in GAP”* by [Wilf Wilson](http://wilf.me)<br />
*“[GAP in Algebraic Research](https://lbfm-rwth.github.io/gap-in-algebraic-research-2018/)” summer school*<br />
19<sup>th</sup> to 22<sup>nd</sup> November 2018<br />
https://github.com/wilfwilson/semigroups-in-algebraic-research
([Link to Binder](https://mybinder.org/v2/gh/wilfwilson/semigroups-in-algebraic-research/master))

# Computing with semigroups in GAP

## What is a semigroup?

A *semigroup* is simply a set with an associative binary operation.

What is associativity? As I'm sure you know, a binary operation $\ast$ on a set $S$ is *associative* if:

$$(x \ast y) \ast z = x \ast (y \ast z)$$

for all elements $x, y, z \in S$.

GAP itself contains long-standing support for semigroups; this is extended by several packages, which I will talk about later. In this notebook, I will focus on some of the functionality that is built into GAP.

According to GAP, what is a semigroup?

In [1]:
IsSemigroup;

<Filter "(IsMagma and IsAssociative)">

Okay, we know what associativity is... but what is a magma? A *magma* is a set with any old binary operation accessible via `*`. So it makes sense that, in GAP, a semigroup in is an associative magma.

What other ‘properties’ does a semigroup inherently have, in GAP?

In [2]:
ShowImpliedFilters(IsSemigroup);

Implies:
   IsListOrCollection
   IsCollection
   IsDuplicateFree
   IsExtLElement
   CategoryCollections(IsExtLElement)
   IsExtRElement
   CategoryCollections(IsExtRElement)
   CategoryCollections(IsMultiplicativeElement)
   IsGeneralizedDomain
   IsMagma
   IsAssociative
   IsGeneratorsOfSemigroup


May imply with:
+IsMagmaWithOne
   IsMonoidAsSemigroup

+IsMagmaWithInverses
   IsSimpleSemigroup
   IsRegularSemigroup
   IsInverseSemigroup
   IsCompletelyRegularSemigroup
   IsGroupAsSemigroup
   IsMonoidAsSemigroup
   IsOrthodoxSemigroup
   IsBlockGroup
   IsLeftSimple
   IsRightSimple
   IsSemigroupWithCommutingIdempotents

+IsNonTrivial
+IsFinite
+IsExtLSet
+IsAssociativeLOpDProd
+IsAssociativeLOpEProd
+IsDistributiveLOpDSum
+IsDistributiveLOpESum
+IsTrivialLOpEOne
+IsLeftActedOnByRing
+IsLeftActedOnByDivisionRing
+IsMagmaWithOne
+IsMagmaWithInversesIfNonzero
+IsNearAdditiveMagma
+IsNearAdditiveGroup
+IsAdditivelyCommutative
+IsLDistributive
+IsRDistributive
+IsEuclideanRing
   IsC

Okay then, what is a group, according to GAP?

In [3]:
IsGroup;

<Filter "(IsMagmaWithInverses and IsAssociative)">

**Great! But we need to be careful...**

This is a slightly technical definition. Basically, what it means is that, according to GAP, a magma $G$ satisfies `IsGroup` if it is mathematically a group, **and moreover**, if $x \in G$, then `x^0` gives the identity of $G$, and `x^-1` gives the group-theoretic inverse of the element $x$ in $G$.

In other words, a group satisfies `IsGroup` if you can calculate the identity and inverse of an element without doing *any* calculations in the group.

Perhaps counter-intuitively, this means that a semigroup in GAP might mathematically define a group, but **not** satisfy `IsGroup`, if it does not support the function `^0` or `^-1`.

Let's see an example of a semigroup that is mathematically a group, but does not satisfy `IsGroup`. It is a semigroup consisting of a single non-invertible $2 \times 2$ matrix over $\mathbb{F}_{2}$.

In [5]:
x := [[1, 1], [0, 0]] * One(GF(2));
S := Semigroup(x);

[ [ Z(2)^0, Z(2)^0 ], [ 0*Z(2), 0*Z(2) ] ]

<commutative semigroup with 1 generator>

In [6]:
IsTrivial(S);

true

Since $S$ is semigroup with only one element, it therefore defines a group (the trivial group). But since `^-1` does not work for its element, it does not satisfy `IsGroup`:

In [8]:
IsGroup(S);
x ^ -1;

false

fail

*This should not be an issue in this mini-course.*

Let's look at some of what is implied by `IsGroup`:

In [9]:
ShowImpliedFilters(IsGroup);

Implies:
   IsListOrCollection
   IsCollection
   IsDuplicateFree
   IsExtLElement
   CategoryCollections(IsExtLElement)
   IsExtRElement
   CategoryCollections(IsExtRElement)
   CategoryCollections(IsMultiplicativeElement)
   CategoryCollections(IsMultiplicativeElementWithOne)
   CategoryCollections(IsMultiplicativeElementWithInverse)
   IsGeneralizedDomain
   IsMagma
   IsMagmaWithOne
   IsMagmaWithInversesIfNonzero
   IsMagmaWithInverses
   IsAssociative
   HasMultiplicativeNeutralElement
   IsGeneratorsOfSemigroup
   IsSimpleSemigroup
   IsRegularSemigroup
   IsInverseSemigroup
   IsCompletelyRegularSemigroup
   IsGroupAsSemigroup
   IsMonoidAsSemigroup
   IsOrthodoxSemigroup
   IsBlockGroup
   IsLeftSimple
   IsRightSimple
   IsSemigroupWithCommutingIdempotents


May imply with:
+IsFinite
   IsComponentObjectRep
   CategoryCollections(IsFiniteOrderElement)
   IsCompletelySimpleSemigroup
   IsFinitelyGeneratedGroup
   IsSubsetLocallyFiniteGroup
   HasIsInfiniteAbelianizationGroup
 

Therefore, mathematically and in GAP, everything that satisfies `IsGroup` also satisfies `IsSemigroup`. That's a relief, at least.

Mathematically, perhaps the “easiest” semigroup to define is the empty semigroup, $\varnothing$. This is also our first example of a semigroup that is not a group. But for technical reasons, this is a difficult semigroup to handle computationally! 

In this mini-course, we will ignore the empty semigroup, and avoid creating it.

## Constructing semigroups from multiplication tables

One of the most basic ways of constructing a finite semigroup in GAP is by specifying its multiplication table.

If a semigroup has $n$ elements, then we can name the elements with the numbers $\{1, \ldots, n\}$ (i.e. we can enumerate them). The multiplication table is then a list `mult` of $n$ lists of length $n$, where `mult[i][j]` stores the value of $i \ast j$ in the semigroup.

For example, the following multiplication table defines a semigroup with four elements.
$$
\begin{array}{l|rrrr}
  \ast & 1 & 2 & 3 & 4 \\
  \hline
  1 & 1 & 2 & 3 & 4 \\
  2 & 2 & 1 & 3 & 4 \\
  3 & 3 & 4 & 3 & 4 \\
  4 & 4 & 3 & 3 & 4
\end{array}
$$

**How do we check that this magma is associative?** It's certainly not immediately obvious! We'll ignore that for now.

In a Cayley tables for a group, every row and every column contains each element of the group exactly once: the multiplication table gives a Latin square.

This is not the case for semigroups in general.

Here, $2$ occurs twice in the second row, and $4$ is the only entry in the first column. Therefore the semigroup is **definitely** not a group. (We're still taking it on faith that it defines a semigroup).

Let's create this table in GAP, as a list-of-lists:

In [10]:
mult := [
[1, 2, 3, 4],
[2, 1, 3, 4],
[3, 4, 3, 4],
[4, 3, 3, 4]
];;

In [11]:
Display(mult);

[ [  1,  2,  3,  4 ],
  [  2,  1,  3,  4 ],
  [  3,  4,  3,  4 ],
  [  4,  3,  3,  4 ] ]


We can create a semigroup object (to which we can apply semigroup-related methods in GAP) from a multiplication table by using the command `SemigroupByMultiplicationTable`.

In [12]:
S := SemigroupByMultiplicationTable(mult);

<semigroup of size 4, with 4 generators>

In [13]:
Size(S);

4

*Note that `SemigroupByMultiplicationTable` incorporates a test for associativity: if we give it a multiplication table that does not define an associative magma, then GAP returns `fail`...*

In [14]:
SemigroupByMultiplicationTable([[1, 1], [2, 1]]);

fail

#### Let's introduce some other semigroup-theoretic notions

Back to our previous semigroup $S$, with $4$ elements. The fact that the first row and column of the multiplication table give the same entries as their labels indicates that $1$ is an *identity element* for the semigroup that it defines. In other words, this multiplication table defines a *monoid* that is not a group.

**Not every semigroup contains an identity element!**

In [15]:
identity := MultiplicativeNeutralElement(S);;

In [16]:
ForAll(S, s -> identity * s = s and s * identity = s);

true

A semigroup $S$ is *commutative* if, for all $x, y \in S$, $xy = yx$. A commutative group is more commonly called *abelian*, but we don't use this name for semigroups.

In [17]:
IsCommutative(S);

false

An element $x$ of a semigroup is *idempotent* if $x^2 = x$.

A group contains exactly one idempotent (its identity element). A finite semigroup contains at least one idempotent. For some semigroups, every element is an idempotent (such a semigroup is called a *band*); and it is possible to have any number of idempotents in between.

3 of the 4 elements in our semigroup $S$ are idempotent:

In [18]:
Number(S, s -> s ^ 2 = s);

3

Or, using the built-in GAP function `IsIdempotent`:

In [19]:
Number(S, IsIdempotent);

3

A semigroup $S$ is *regular* if, for all $x \in S$, there exists $y \in S$ such that $x = xyx$. Every group is regular: set $y = x^{-1}$.

In [20]:
IsRegularSemigroup(S);

true

A semigroup $S$ is *inverse* if, for all $x \in S$, there exists a **unique** $y \in S$ such that $x = xyx$ and $y = yxy$. This unique element $y$ is often written as $x^{-1}$.

**Note that `x * x^-1` does not necessary give you the identity element – in fact, an inverse semigroup does not even need to possess an identity!**

In [21]:
IsInverseSemigroup(S);

false

**Roughly, to give you a bit of intuition**:

* Groups capture structure-perserving bijections on a structure (automorphisms/symmetries);
* Inverse semigroups capture structure-preserving bijections between substructures of a structure (partial automorphisms/symmetries);
* Semigroups in general capture arbitrary structure-preserving functions on a structure (endomorphisms).

**For instance, if the structure is a group $G$:**

* We use a group to capture the automorphisms of $G$;
* We could use an inverse subsemigroup to capture the isomorphisms between subgroups of $G$;
* We could use a semigroup to capture the homomorphisms from $G$ to itself.

#### Downsides of multiplication tables

We almost never use multiplication tables:

* They are tedious and very fiddly to type into a computer.
* They use a lot of memory: roughly $n^2 * log_{2}n$ bits. This can get too big very easily.
* It takes time to verify that a multiplication table describes an associative operation.
* For practical reasons, it is only possible to create finite semigroups this way!

## Constructing semigroups by generators

By using a generating set, you can often specify a semigroup $S$ by using many fewer elements than $S$ contains in total.

For basically any algorithm, the complexity is given in terms of the number of generators of the semigroup (amongst other things). **The smaller the generating set, the better**.

A lot of effort goes into describing ”small” generating sets of semigroups. A group $G$ requires at most $\log_{2}|G|$ generators; a semigroup $S$ may require $|S|$ generators.

**What kind of generators do we use in GAP?**

### Abstract: finitely-presented semigroups

You can create finitely-generated free semigroups and quotients by finitely generated congruences (finitely-presented semigroups) in GAP, similarly to finitely-presented groups, but I will not talk about them further.

In [22]:
F := FreeSemigroup("a", "b");

<infinite semigroup with 2 generators>

In [24]:
a := F.1; b := F.2;

<object>

<object>

In [25]:
S := F / [[a ^ 3, b ^ 2], [a * b * a, b ^ 2 * a]];

<semigroup with 2 generators>

In [26]:
Size(S);

15

See the `kbmag` package for GAP by Derek Holt for additional functionality for fp-semigroups and monoids:

https://gap-packages.github.io/kbmag

### Transformation semigroups

Permutations are to groups as transformations are to semigroups.

A *permutation* of degree $n$ is a bijective function from $\{1, \ldots, n\}$ to itself (i.e. an *automorphism* of $\{1, \ldots, n\}$).

A *transformation* of degree $n$ is any old function from $\{1, \ldots, n\}$ to itself (i.e. an *endomorphism* of $\{1, \ldots, n\}$).

Possibly you were first introduced to permutations via two-line notation. The following is one way of writing the permutation of degree $4$ that reverses the order of $\{1, 2, 3, 4\}$:

$$
\begin{pmatrix}
  1 & 2 & 3 & 4 \\
  4 & 3 & 2 & 1
\end{pmatrix}
$$

It means that 1 maps to 4, 2 maps to 3, 3 maps to 2, and 4 maps to 1. 

If we remove the restriction that the bottom line contains each number in $\{1, \ldots, n\}$ exactly once, then we get the transformations. For example:

$$
x = \begin{pmatrix}
  1 & 2 & 3 & 4 & 5 \\
  3 & 3 & 3 & 3 & 3
\end{pmatrix}
$$

is the constant transformation of degree $5$ with image $3$, and

$$
y = \begin{pmatrix}
  1 & 2 & 3 & 4 & 5 \\
  1 & 2 & 1 & 4 & 1
\end{pmatrix}
$$

is the transformation of degree $5$ corresponding to $x \mapsto \textrm{gcd}(x, 4)$.

We can create a transformation in GAP by listing its bottom line when it is written in two-line notation:

In [27]:
x := Transformation([3, 3, 3, 3, 3]);

Transformation( [ 3, 3, 3, 3, 3 ] )

In [28]:
y := Transformation([1, 2, 1, 4, 1]);

Transformation( [ 1, 2, 1, 4, 1 ] )

As with permutations, you can compose transformations, and this composition is associative. (We compose from left to right in GAP). Therefore you can make semigroups of transformations.

You can make transformation semigroups in GAP by using the command `Semigroup`:

In [29]:
S := Semigroup(x, y);

<transformation semigroup of degree 5 with 2 generators>

In [30]:
Size(S);

3

In [31]:
Difference(S, [x, y]);

[ Transformation( [ 1, 1, 1, 1, 1 ] ) ]

So here, our generators `x` and `y` only generate one additional element.

#### Brief detour

In fact, you can use the command `Semigroup` to create a semigroup out of almost any set of elements for which there is an associative operation defined via the function `*`. For instance: matrices in GAP, binary relations, general mappings (with equal `Source` and `Range`)...

Specifically, your list of generators must satisfy `IsGeneratorsOfSemigroup`.

In [32]:
relations := [
BinaryRelationOnPoints([[1, 3], [2], [3]]),
BinaryRelationOnPoints([[2], [2], [1, 3]])
];

[ <object>, <object> ]

In [33]:
IsGeneratorsOfSemigroup(relations);

true

In [34]:
T := Semigroup(relations);

<semigroup with 2 generators>

In [35]:
Size(T);

5

#### Let's randomly generate some transformation semigroups

You can use the GAP function `RandomTransformation(deg)` to create a random transformation of degree `deg`. By calling this repeatedly, we can create a “random” generating set, and hence, a “random” semigroup.

In [36]:
random_trans_semigroup := function(deg, k)
    local gens;
    gens := List([1 .. k], i -> RandomTransformation(deg));
    return Semigroup(gens);
end;

function( deg, k ) ... end

In [37]:
S := random_trans_semigroup(6, 4);

<transformation semigroup of degree 6 with 4 generators>

In [38]:
Size(S);

3699

In [39]:
T := random_trans_semigroup(8, 2);

<transformation semigroup of degree 8 with 2 generators>

In [40]:
Size(T);

458

#### The full transformation semigroup/monoid

The semigroup (indeed, it's a monoid) of all transformations of degree $n$, with the operation of composition of functions, is called the *full transformation semigroup (or monoid) of degree $n$*, written $T_{n}$. You can use both `FullTransformationSemigroup` and `FullTransformationMonoid` to do create it in GAP.

In [41]:
S := FullTransformationSemigroup(5);

<full transformation monoid of degree 5>

In [42]:
S := FullTransformationMonoid(6);

<full transformation monoid of degree 6>

#### Analogue of Cayley's theorem for semigroups

**Cayley's theorem for groups:** every group $G$ embeds into $S_{|G|}$.

Idea (right regular action): we associate a number to each $g \in G$. Then, for each $g \in G$, the permutation in $S_{|G|}$ corresponding to $g$ is the one that maps $x$ to $x * g$, for all $x \in G$.

**For semigroups:** every semigroup $S$ embeds into $T_{|S| + 1}$.

Idea: we associate a number to each $s \in S$. Then, for each $s \in S$, the transformation in $T_{|S| + 1}$ corresponding to $s$ is the one that maps $x$ to $x * s$, for all $x \in S$, **and it maps a new element $\Omega$ to $s$**.

If GAP has no better ideas, then it uses this idea when you call `IsomorphismTransformationSemigroup` for a finite semigroup.


In [43]:
S := SemigroupByMultiplicationTable([[1, 1], [2, 2]]);

<semigroup of size 2, with 2 generators>

In [44]:
iso := IsomorphismTransformationSemigroup(S);;

In [45]:
T := Range(iso);

<transformation semigroup of size 2, degree 3 with 2 generators>

**Being able to find small degree transformation representations of semigroups would be really nice! But it's difficult.** Some of the exercises will be related to this topic.

### Semigroups of partial permutations (or partial perms)

Permutations are to groups as partial permutations are to inverse semigroups.

A *partial permutation* of degree $n$ is any **injective, partial** function from $\{1, \ldots, n\}$ to itself (i.e. any partial automorphism of $\{1, \ldots, n\}$.

When written in two-line notation, the bottom line should contain no repeats, but not every space needs to be completed. We use a dash "–" in this notation to indicate that the image of a point is undefined. For example:

$$
x = \begin{pmatrix}
  1 & 2 & 3 & 4 & 5 \\
  5 & - & 1 & 2 & -
\end{pmatrix}
$$

is the partial permutation mapping $1 \to 5$, and $3 \to 1$, and $4 \to 2$, and where $2$ and $5$ are not in the domain.

You can create a partial perm in GAP in two ways: firstly, by listing the bottom line; using the integer `0` to denote 'undefined':

In [46]:
x1 := PartialPerm([5, 0, 1, 2, 0]);

[3,1,5][4,2]

Secondly, you can create a partial perm in GAP by listing the domain (in ascending order), and then listing, in order, in the image of each point in the domain:

In [47]:
x2 := PartialPerm([1, 3, 4], [5, 1, 2]);

[3,1,5][4,2]

In [48]:
x1 = x2;

true

#### The symmetric inverse semigroup/monoid

The semigroup of all partial perms of degree $n$ is called the symmetric inverse semigroup (or monoid).

In [49]:
S := SymmetricInverseSemigroup(4);

<symmetric inverse monoid of degree 4>

In [50]:
IsInverseSemigroup(S);

true

In [51]:
T := SymmetricInverseMonoid(5);

<symmetric inverse monoid of degree 5>

In [52]:
IsInverseSemigroup(T);

true

In [53]:
Size(T);

1546

There is an analogue of Cayley's theorem which states that every inverse semigroup $S$ is isomorphic to an inverse subsemigroup of the symmetric inverse monoid of degree $|S|$.

So: all inverse semigroups can be represented as semigroups of partial perms. The inverse in the semigroup is the obvious inverse of the function:

In [54]:
x1 ^ -1;

[2,4][5,1,3]

In [55]:
x2 ^ -1;

[2,4][5,1,3]

However, it is also possible to create semigroups of partial permutations that are *not* inverse:

In [56]:
S := Semigroup(PartialPerm([2, 4, 0, 1, 3]));

<commutative partial perm semigroup of rank 4 with 1 generator>

In [57]:
IsInverseSemigroup(S);

false

# Exercises

### Multiplication tables

1. For each $n \in \mathbb{N}$, how many binary operations are there on a set of $n$ elements? Equivalently, how many $n \times n$ multiplication tables are there?

2. Construct all $3 \times 3$ multiplication tables in GAP, where each multiplication table is a list of $3$ lists of length $3$, whose entries are from $\{1, 2, 3\}$. If you don't want to do this, use the following code:

In [60]:
all_tables := function(n)
  local table, tables, iter, choice;
  tables := [];
  iter := IteratorOfTuples([1 .. n], n * n);
  for choice in iter do
    table := List([0, 1 .. n - 1],
                  i -> choice{[(n * i) + 1 .. (n * (i + 1))]});
    Add(tables, table);
  od;
  return tables;
end;
all_tables_3 := all_tables(3);;
Length(all_tables_3);

function( n ) ... end

19683

3. How many define semigroups?

4. How many define commutative semigroups?

5. How many define monoids?

6. How many define bands? (A band is a semigroup in which every element is idempotent.)

7. How many define regular semigroups?

### Transformation and partial perm semigroups

1. How many elements does the full transformation monoid of degree $n$ contain? Equivalently, how many transformations are there of degree $n$?

2. How many elements does the symmetric inverse monoid of degree $n$ contain? Equivalently, how many partial permutations are there of degree $n$?

3. When doing the analogue of Cayley's theorem for semigroups, why did we need to invent this new element $\Omega$?  What if we just did the same thing as for groups?

    * Write a function `f` which takes a multiplication table `tab` and an element `s` in the semigroup, and returns a transformation such that `f(tab, s)` maps $x$ to $x * s$ (i.e. `x` is mapped to `tab[x][s]`), for all $x \in S$.

    * Find some semigroups with three elements (using the multiplication tables from above) where this function is **not** injective, and therefore not an isomorphism.

4. Write a function that takes a multiplication table defining a semigroup $S$, and returns a transformation semigroup that is isomorphic to $S$, using the analogue of Cayley's theorem.

5. Write a function that takes a `SemigroupByMultiplicationTable`, called `S`, and returns an *isomorphism* to a transformation semigroup `T`, using `MagmaHomomorphismByFunctionsNC`. To do this, the first argument to `MagmaHomomorphismByFunctionsNC` should be `S`, the second should be `T`, and the third argument should be a GAP function that takes an element of `S` to its corresponding element of `T`. (If you really want to over-achieve, look at the GAP code for `MagmaIsomorphismByFunctionsNC` and use that).